# Personal Finance EDA

In [1]:
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [2]:
df = pd.read_csv('./data/personal_transactions.csv')
df['Date'] = pd.to_datetime(df['Date'])

In [3]:
df.head()

,Date,Description,Amount,Transaction Type,Category,Account Name
0,2018-01-01,Amazon,11.11,debit,Shopping,Platinum Card
1,2018-01-02,Mortgage Payment,1247.44,debit,Mortgage & Rent,Checking
2,2018-01-02,Thai Restaurant,24.22,debit,Restaurants,Silver Card
3,2018-01-03,Credit Card Payment,2298.09,credit,Credit Card Payment,Platinum Card
4,2018-01-04,Netflix,11.76,debit,Movies & DVDs,Platinum Card


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 806 entries, 0 to 805
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Date              806 non-null    datetime64[ns]
 1   Description       806 non-null    object        
 2   Amount            806 non-null    float64       
 3   Transaction Type  806 non-null    object        
 4   Category          806 non-null    object        
 5   Account Name      806 non-null    object        
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 37.9+ KB


## Debits Analysis

In [5]:
debits = df[df["Transaction Type"] == 'debit']

In [6]:
def count_sum(data, column: str, plot="Pie"):
    by_column = data\
        .groupby(column)\
        .agg({"Transaction Type": "count", "Amount": "sum"})\
        .rename(columns={"Transaction Type": "Total"})\
        .reset_index()
    by_column.columns = [column, "Total", "Sum"]

    labels = by_column[column]

    fig = None

    if plot == 'Pie':
        fig = make_subplots(1, 2, specs=[[{'type': 'domain'}, {'type': 'domain'}]],
                            subplot_titles=['Sum', 'Total'])
        fig.add_trace(
            go.Pie(
                labels=labels,
                values=by_column["Sum"],
                name="Sum"
            ),
            1, 1)
        fig.add_trace(
            go.Pie(
                labels=labels,
                values=by_column["Total"],
                name="Total"
            ),
            1, 2)
    elif plot == 'Scatter':
        fig = make_subplots(1, 2,
                            subplot_titles=['Sum', 'Total'])
        fig.add_trace(
            go.Scatter(
                x=labels,
                y=by_column["Sum"],
                name="Sum"
            ),
            1, 1)
        fig.add_trace(
            go.Scatter(
                x=labels,
                y=by_column["Total"],
                name="Total"
            ),
            1, 2)

    fig.update_layout(title_text=f"{column} Analysis")
    fig.show()

    return by_column

### Debits by Category?

In [7]:
gastos_by_cat = count_sum(debits, "Category")
gastos_by_cat

,Category,Total,Sum
0,Alcohol & Bars,25,539.13
1,Auto Insurance,18,1350.00
2,Coffee Shops,31,115.54
3,Credit Card Payment,71,33041.36
4,Electronics & Software,4,719.00
5,Entertainment,1,9.62
6,Fast Food,16,330.63
7,Food & Dining,2,77.75
8,Gas & Fuel,52,1715.17
9,Groceries,105,2795.21


### Debits by Payment Account?

In [8]:
gastos_by_payment_way = count_sum(debits, "Account Name")
gastos_by_payment_way

,Account Name,Total,Sum
0,Checking,218,82498.14
1,Platinum Card,324,8996.31
2,Silver Card,146,4589.33


### Total $ Expenses per date

In [9]:
gastos_by_date = count_sum(debits, "Date", plot="Scatter")
gastos_by_date

,Date,Total,Sum
0,2018-01-01,1,11.11
1,2018-01-02,2,1271.66
2,2018-01-04,1,11.76
3,2018-01-05,1,25.85
4,2018-01-06,1,18.45
...,...,...,...
404,2019-09-19,2,43.56
405,2019-09-22,1,131.10
406,2019-09-23,3,61.77
407,2019-09-28,2,37.73


## Income Analysis

In [10]:
credits = df[df["Transaction Type"] == 'credit']

### Income by Account?

In [11]:
income_by_account = count_sum(credits, "Account Name")
income_by_account

,Account Name,Total,Sum
0,Checking,46,93750.00
1,Platinum Card,42,21139.93
2,Silver Card,30,9379.83
